### 1. Import Dependencies

In [125]:
# Import gym stuff
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

# Import helpers
import numpy as np
import random
import os

# import stable baselines stuff
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

### 2. Types of Spaces

In [7]:
print(Discrete(3))
Discrete(3).sample()

Discrete(3)


2

In [20]:
print(Box(0, 1, shape=(3, 3)).sample()) # (low value, high value, shape)

[[0.9018345  0.7179776  0.65608484]
 [0.24972779 0.3431002  0.27227515]
 [0.8832656  0.04279574 0.6032188 ]]


In [23]:
# combine discrete and continuous space
print(Tuple((Discrete(3), Box(0, 1, shape=(3, )))))
print(Tuple((Discrete(3), Box(0, 1, shape=(3, )))).sample())

Tuple(Discrete(3), Box(0.0, 1.0, (3,), float32))
(2, array([0.06864689, 0.5716351 , 0.82408226], dtype=float32))


In [25]:
print(Dict({'height': Discrete(2), 'speed': Box(0, 100, shape=(1, ))}))
print(Dict({'height': Discrete(2), 'speed': Box(0, 100, shape=(1, ))}).sample())

Dict('height': Discrete(2), 'speed': Box(0.0, 100.0, (1,), float32))
OrderedDict([('height', 1), ('speed', array([83.73064], dtype=float32))])


In [30]:
print(MultiBinary(4))
print(MultiBinary(4).sample())

MultiBinary(4)
[1 1 1 0]


In [58]:
print(MultiDiscrete([5, 2, 2]))
print(MultiDiscrete([5, 2, 2]).sample())

MultiDiscrete([5 2 2])
[0 1 0]


### 3. Building an environment
- Build an agent to give us the best shower possible
- Random temperature
- 37 and 39 degrees 

In [108]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=np.array([0]), high=np.array([100]), shape=(1, ))
        self.state = 38 + random.randint(-3, 3)
        self.shower_length = 60
        
    def step(self, action):
        # apply temp adj
        self.state += action - 1

        # decrease shower time
        self.shower_length -= 1

        # calculate reward
        if self.state >= 37 and self.state <= 39:
            reward = 1
        else:
            reward = -1

        if self.shower_length <= 0:
            done = True
        else:
            done = False

        info = {}

        return self.state, reward, done, info
        
    def render(self):
        # implement viz
        pass
    def reset(self):
        self.state = np.array([38 + random.randint(-3, 3)]).astype(float)
        self.shower_length = 60
        return self.state


In [109]:
env = ShowerEnv()
print(env.observation_space)
print(env.action_space)
print(env.observation_space.sample())
print(env.action_space.sample())

Box(0.0, 100.0, (1,), float32)
Discrete(3)
[83.317566]
2


/home/aaron/rl_tutorial/lib/python3.8/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [111]:
print(env.step(env.action_space.sample()))

(43, -1, False, {})


### 4. Test Environment

In [115]:
episodes = 10

for episode in range(1, episodes+1):
	obs = env.reset()
	done = False
	score = 0
 
	while not done:
		env.render()
		action = env.action_space.sample()
		obs, reward, done, info = env.step(action)
		score += reward
	print(f'Episode: {episode}, Score: {score}')
env.close()

Episode: 1, Score: -44
Episode: 2, Score: -6
Episode: 3, Score: -20
Episode: 4, Score: -60
Episode: 5, Score: -24
Episode: 6, Score: -58
Episode: 7, Score: -12
Episode: 8, Score: -48
Episode: 9, Score: -36
Episode: 10, Score: -28


### 5. Train model

In [116]:
log_path = os.path.join('training', 'logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/aaron/rl_tutorial/lib/python3.8/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [119]:
model.learn(total_timesteps=100000)

Logging to training/logs/PPO_13
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -1       |
| time/              |          |
|    fps             | 1475     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | 1.26         |
| time/                   |              |
|    fps                  | 1010         |
|    iterations           | 2            |
|    time_elapsed         | 4            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0018426867 |
|    clip_fraction        | 0.0309       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.393       |
|    explained_variance   

### 6. Save model

In [120]:
shower_path = os.path.join('training', 'saved_models', 'shower_model')

In [121]:
model.save(shower_path)

In [122]:
del model

In [123]:
model = PPO.load(shower_path, env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/aaron/rl_tutorial/lib/python3.8/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [129]:
evaluate_policy(model, env, n_eval_episodes=10)

(12.0, 58.787753826796276)